In [10]:
import numpy as np
import pennylane as qml
from qiskit.quantum_info import SparsePauliOp, Pauli
from susy_qm import calculate_Hamiltonian, calculate_Hamiltonian2

In [27]:
cutoff = 8
potential = 'AHO'
H = calculate_Hamiltonian(cutoff, potential)

In [28]:
SparsePauliOp.from_operator(H)

SparsePauliOp(['IIII', 'IIXI', 'IIXZ', 'IIZI', 'IIZZ', 'IXII', 'IXIZ', 'IXXI', 'IXXZ', 'IXZI', 'IXZZ', 'IYYI', 'IYYZ', 'IZII', 'IZIZ', 'IZXI', 'IZXZ', 'IZZI', 'IZZZ', 'ZIII', 'ZIXI', 'ZIXZ', 'ZIZI', 'ZIZZ', 'ZXXI', 'ZXXZ', 'ZYYI', 'ZYYZ', 'ZZII', 'ZZIZ', 'ZZXI', 'ZZXZ', 'ZZZI', 'ZZZZ'],
              coeffs=[ 1.13312500e+02+0.j,  8.81890635e+01+0.j, -7.15058155e-02+0.j,
 -9.06250000e+00+0.j, -2.50625000e+01+0.j,  2.80566466e+01+0.j,
 -6.76713867e+00+0.j,  4.21567483e+01+0.j, -1.41466758e+01+0.j,
 -1.65897126e+01+0.j,  1.11774278e+00+0.j,  3.90996929e+01-0.j,
 -1.27666714e+01+0.j, -8.05625000e+01+0.j, -1.38125000e+01+0.j,
 -7.40404151e+01+0.j, -7.97834659e+00+0.j, -1.66875000e+01+0.j,
  3.40625000e+01+0.j,  5.75000000e+00+0.j,  2.96656305e+00+0.j,
 -3.82273369e-01+0.j, -7.50000000e-01+0.j, -7.50000000e-01+0.j,
  1.48804454e+00+0.j, -1.89006439e-01+0.j,  1.48804454e+00-0.j,
 -1.89006439e-01+0.j, -2.25000000e+00+0.j, -7.50000000e-01+0.j,
 -1.51767431e+00+0.j, -5.95519820e-03+0.j, -7.50000

In [29]:
qml.pauli_decompose(H)

(
    113.31249999999994 * (I(0) @ I(1) @ I(2) @ I(3))
  + 88.18906352189643 * (I(0) @ I(1) @ X(2) @ I(3))
  + -0.07150581550838098 * (I(0) @ I(1) @ X(2) @ Z(3))
  + -9.062500000000004 * (I(0) @ I(1) @ Z(2) @ I(3))
  + -25.06249999999999 * (I(0) @ I(1) @ Z(2) @ Z(3))
  + 28.056646641385136 * (I(0) @ X(1) @ I(2) @ I(3))
  + -6.767138672101423 * (I(0) @ X(1) @ I(2) @ Z(3))
  + 42.156748329868826 * (I(0) @ X(1) @ X(2) @ I(3))
  + -14.146675794569653 * (I(0) @ X(1) @ X(2) @ Z(3))
  + -16.589712610811898 * (I(0) @ X(1) @ Z(2) @ I(3))
  + 1.1177427806382285 * (I(0) @ X(1) @ Z(2) @ Z(3))
  + 39.09969291964405 * (I(0) @ Y(1) @ Y(2) @ I(3))
  + -12.76667136746972 * (I(0) @ Y(1) @ Y(2) @ Z(3))
  + -80.56249999999994 * (I(0) @ Z(1) @ I(2) @ I(3))
  + -13.8125 * (I(0) @ Z(1) @ I(2) @ Z(3))
  + -74.04041513104318 * (I(0) @ Z(1) @ X(2) @ I(3))
  + -7.978346587610893 * (I(0) @ Z(1) @ X(2) @ Z(3))
  + -16.68749999999999 * (I(0) @ Z(1) @ Z(2) @ I(3))
  + 34.062499999999986 * (I(0) @ Z(1) @ Z(2) @ Z(3))

In [18]:
def pennylane_to_qiskit_sparse_pauliop(pl_hamiltonian, wire_order=None):
    """Convert a PennyLane Hamiltonian into a Qiskit SparsePauliOp with full precision."""
    if wire_order is None:
        wire_order = sorted(pl_hamiltonian.wires.tolist())

    pauli_labels = []
    coeffs = []

    for coeff, op in zip(pl_hamiltonian.coeffs, pl_hamiltonian.ops):
        label = ['I'] * len(wire_order)

        # Handle tensor products
        ops = [op] if not hasattr(op, "obs") else op.obs
        for subop in ops:
            wire = subop.wires[0]
            wire_index = wire_order.index(wire)
            label[wire_index] = subop.name[-1].upper()  # ensure uppercase

        pauli_labels.append("".join(label))
        coeffs.append(np.complex128(coeff))  # force high precision

    return SparsePauliOp.from_list(list(zip(pauli_labels, coeffs)), dtype=np.complex128)

In [20]:
pennylane_to_qiskit_sparse_pauliop(qml.pauli_decompose(H))

SparsePauliOp(['YYY', 'YXY', 'YXZ', 'YZY', 'YZZ', 'ZYY', 'ZXY', 'ZXZ', 'ZZY', 'ZZZ'],
              coeffs=[10.3125    +0.j,  6.88222151+0.j, -1.8440857 +0.j, -4.0625    +0.j,
 -4.0625    +0.j,  2.75      +0.j,  1.44888874+0.j, -0.38822857+0.j,
 -0.75      +0.j, -0.75      +0.j])

In [2]:
cutoff = 4
potential = 'DW'
m=1
g=1
u=1

In [3]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential, m, g, u)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]

In [4]:
eigenvalues.real

array([0.90655987, 0.95063353, 1.69566635, 1.83552815])

In [5]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian2(cutoff, potential, m, g, u)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]

In [6]:
eigenvalues.real

array([0.90655987, 0.95063353, 1.69566635, 1.83552815])